In [3]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')

In [15]:
# path='D:\Python\Elo'
# df_train=pd.read_csv(os.path.join(path,'main.csv'))[['card_id','target']]
# path='D:\Python\Elo\Stack'
# print(os.listdir(path))
# for name in os.listdir(path):
#     if 'PROB' in name and 'prediction' in name:
#         df_train=df_train.merge(pd.read_csv(os.path.join(path,name),names=['card_id',name[11:-3]]),how='left',on='card_id')

path='D:\Python\Elo\Stack'
df_train=pd.DataFrame()
print(os.listdir(path))
for name in os.listdir(path):
    if 'prediction' in name and 'PROB' in name:
        df_train[name[11:-3]]=pd.read_csv(os.path.join(path,name))['target']
path='D:\Python\Elo'
df_train['target']=pd.read_csv(os.path.join(path,'main.csv'))['target']

# df_train['card_id']=pd.read_csv(os.path.join(path,'main.csv'))['card_id']
# data = pd.read_csv(os.path.join(path,'result.csv'))
# selector=['hist_month_lag_mean', 'hist_month_diff_mean',
#        'hist_fathers_day_2017_mean', 'hist_duration_min',
#        'hist_purchase_date_diff', 'hist_purchase_date_uptonow',
#        'hist_last_installments_sum_0', 'hist_purchase_amount_mean_-4',
#        'new_purchase_mean_last_3month_mean','card_id']
# df_train=df_train.merge(data[selector],how='left',on='card_id')

['prediction_LGB_6part_3.66.csv', 'prediction_LGB_7part_3.65.csv', 'prediction_LGB_7part_3.66.csv', 'prediction_LGB_8part_3.66.csv', 'prediction_LGB_full_1.55.csv', 'prediction_LGB_full_3.64544.csv', 'prediction_LGB_full_3.64573.csv', 'prediction_LGB_full_3.64753.csv', 'prediction_LGB_full_3.64904.csv', 'prediction_LGB_half_3.65.csv', 'prediction_LGB_part_1.62.csv', 'prediction_LGB_part_3.72522.csv', 'prediction_LGB_part_3.72907.csv', 'prediction_NN_3.7283.csv', 'prediction_PROB_CAT_7part_0.89.csv', 'prediction_PROB_CAT_8part_0.89.csv', 'prediction_PROB_LGB_6part_0.9.csv', 'prediction_PROB_LGB_7part_0.9.csv', 'prediction_PROB_LGB_half_0.9.csv', 'prediction_PROB_NN_0.8797.csv', 'submission_LGB_6part_3.66.csv', 'submission_LGB_7part_3.65.csv', 'submission_LGB_7part_3.66.csv', 'submission_LGB_8part_3.66.csv', 'submission_LGB_full_1.55.csv', 'submission_LGB_full_3.64544.csv', 'submission_LGB_full_3.64573.csv', 'submission_LGB_full_3.64753.csv', 'submission_LGB_full_3.64904.csv', 'submissio

In [17]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
skf=StratifiedKFold(5,shuffle=True,random_state=100)
val_pr=np.zeros(len(df_train))
feature_importance_df = pd.DataFrame()
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    pool=Pool(df_train[df_train_columns].loc[train_index],df_train['outliers'].loc[train_index])
    val_pool=Pool(df_train[df_train_columns].loc[val_index],df_train['outliers'].loc[val_index])
    num_round = 10000
    model = CatBoostClassifier(iterations=2000,learning_rate=0.01, loss_function='CrossEntropy',eval_metric='AUC',early_stopping_rounds=200)
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=True)
    fold_importance_df=pd.DataFrame()
    fold_importance_df['feature'] = df_train_columns
    fold_importance_df['importance']=model.feature_importances_
    feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index],prediction_type='Probability')[:,1]
roc_auc_score(df_train['outliers'],val_pr)

0:	test: 0.8140338	best: 0.8140338 (0)	total: 189ms	remaining: 6m 17s
1:	test: 0.8785552	best: 0.8785552 (1)	total: 287ms	remaining: 4m 46s
2:	test: 0.8811257	best: 0.8811257 (2)	total: 349ms	remaining: 3m 52s
3:	test: 0.8887460	best: 0.8887460 (3)	total: 399ms	remaining: 3m 19s
4:	test: 0.8916203	best: 0.8916203 (4)	total: 464ms	remaining: 3m 5s
5:	test: 0.8938445	best: 0.8938445 (5)	total: 527ms	remaining: 2m 55s
6:	test: 0.8945489	best: 0.8945489 (6)	total: 575ms	remaining: 2m 43s
7:	test: 0.8941619	best: 0.8945489 (6)	total: 623ms	remaining: 2m 35s
8:	test: 0.8943927	best: 0.8945489 (6)	total: 668ms	remaining: 2m 27s
9:	test: 0.8957467	best: 0.8957467 (9)	total: 718ms	remaining: 2m 22s
10:	test: 0.8951763	best: 0.8957467 (9)	total: 782ms	remaining: 2m 21s
11:	test: 0.8951239	best: 0.8957467 (9)	total: 829ms	remaining: 2m 17s
12:	test: 0.8957736	best: 0.8957736 (12)	total: 874ms	remaining: 2m 13s
13:	test: 0.8956358	best: 0.8957736 (12)	total: 940ms	remaining: 2m 13s
14:	test: 0.895

119:	test: 0.8971592	best: 0.8971592 (119)	total: 7.53s	remaining: 1m 58s
120:	test: 0.8971809	best: 0.8971809 (120)	total: 7.58s	remaining: 1m 57s
121:	test: 0.8971902	best: 0.8971902 (121)	total: 7.63s	remaining: 1m 57s
122:	test: 0.8972033	best: 0.8972033 (122)	total: 7.68s	remaining: 1m 57s
123:	test: 0.8972007	best: 0.8972033 (122)	total: 7.73s	remaining: 1m 56s
124:	test: 0.8971971	best: 0.8972033 (122)	total: 7.79s	remaining: 1m 56s
125:	test: 0.8971741	best: 0.8972033 (122)	total: 7.85s	remaining: 1m 56s
126:	test: 0.8971826	best: 0.8972033 (122)	total: 7.89s	remaining: 1m 56s
127:	test: 0.8971922	best: 0.8972033 (122)	total: 7.94s	remaining: 1m 56s
128:	test: 0.8971878	best: 0.8972033 (122)	total: 7.99s	remaining: 1m 55s
129:	test: 0.8971766	best: 0.8972033 (122)	total: 8.04s	remaining: 1m 55s
130:	test: 0.8971813	best: 0.8972033 (122)	total: 8.09s	remaining: 1m 55s
131:	test: 0.8971728	best: 0.8972033 (122)	total: 8.13s	remaining: 1m 55s
132:	test: 0.8971930	best: 0.8972033 (

231:	test: 0.8977053	best: 0.8977129 (222)	total: 12.9s	remaining: 1m 38s
232:	test: 0.8977013	best: 0.8977129 (222)	total: 12.9s	remaining: 1m 37s
233:	test: 0.8977002	best: 0.8977129 (222)	total: 13s	remaining: 1m 37s
234:	test: 0.8976784	best: 0.8977129 (222)	total: 13s	remaining: 1m 37s
235:	test: 0.8976811	best: 0.8977129 (222)	total: 13.1s	remaining: 1m 37s
236:	test: 0.8976800	best: 0.8977129 (222)	total: 13.1s	remaining: 1m 37s
237:	test: 0.8976832	best: 0.8977129 (222)	total: 13.1s	remaining: 1m 37s
238:	test: 0.8976881	best: 0.8977129 (222)	total: 13.2s	remaining: 1m 37s
239:	test: 0.8976816	best: 0.8977129 (222)	total: 13.2s	remaining: 1m 36s
240:	test: 0.8976813	best: 0.8977129 (222)	total: 13.3s	remaining: 1m 36s
241:	test: 0.8976994	best: 0.8977129 (222)	total: 13.3s	remaining: 1m 36s
242:	test: 0.8977045	best: 0.8977129 (222)	total: 13.4s	remaining: 1m 36s
243:	test: 0.8977185	best: 0.8977185 (243)	total: 13.4s	remaining: 1m 36s
244:	test: 0.8977243	best: 0.8977243 (244)

346:	test: 0.8980580	best: 0.8980681 (342)	total: 17.9s	remaining: 1m 25s
347:	test: 0.8980621	best: 0.8980681 (342)	total: 18s	remaining: 1m 25s
348:	test: 0.8980649	best: 0.8980681 (342)	total: 18s	remaining: 1m 25s
349:	test: 0.8980656	best: 0.8980681 (342)	total: 18s	remaining: 1m 25s
350:	test: 0.8980661	best: 0.8980681 (342)	total: 18.1s	remaining: 1m 24s
351:	test: 0.8980744	best: 0.8980744 (351)	total: 18.1s	remaining: 1m 24s
352:	test: 0.8980744	best: 0.8980744 (351)	total: 18.2s	remaining: 1m 24s
353:	test: 0.8980744	best: 0.8980744 (351)	total: 18.2s	remaining: 1m 24s
354:	test: 0.8980734	best: 0.8980744 (351)	total: 18.3s	remaining: 1m 24s
355:	test: 0.8980804	best: 0.8980804 (355)	total: 18.3s	remaining: 1m 24s
356:	test: 0.8980781	best: 0.8980804 (355)	total: 18.3s	remaining: 1m 24s
357:	test: 0.8980784	best: 0.8980804 (355)	total: 18.4s	remaining: 1m 24s
358:	test: 0.8980781	best: 0.8980804 (355)	total: 18.4s	remaining: 1m 24s
359:	test: 0.8980779	best: 0.8980804 (355)	t

460:	test: 0.8982495	best: 0.8982495 (460)	total: 22.8s	remaining: 1m 16s
461:	test: 0.8982502	best: 0.8982502 (461)	total: 22.9s	remaining: 1m 16s
462:	test: 0.8982525	best: 0.8982525 (462)	total: 22.9s	remaining: 1m 16s
463:	test: 0.8982580	best: 0.8982580 (463)	total: 22.9s	remaining: 1m 15s
464:	test: 0.8982575	best: 0.8982580 (463)	total: 23s	remaining: 1m 15s
465:	test: 0.8982677	best: 0.8982677 (465)	total: 23s	remaining: 1m 15s
466:	test: 0.8982681	best: 0.8982681 (466)	total: 23.1s	remaining: 1m 15s
467:	test: 0.8982673	best: 0.8982681 (466)	total: 23.1s	remaining: 1m 15s
468:	test: 0.8982693	best: 0.8982693 (468)	total: 23.2s	remaining: 1m 15s
469:	test: 0.8982822	best: 0.8982822 (469)	total: 23.2s	remaining: 1m 15s
470:	test: 0.8982808	best: 0.8982822 (469)	total: 23.2s	remaining: 1m 15s
471:	test: 0.8982677	best: 0.8982822 (469)	total: 23.3s	remaining: 1m 15s
472:	test: 0.8982767	best: 0.8982822 (469)	total: 23.3s	remaining: 1m 15s
473:	test: 0.8982786	best: 0.8982822 (469)

...

543:	test: 0.8980644	best: 0.8980738 (513)	total: 25.1s	remaining: 1m 7s
544:	test: 0.8980655	best: 0.8980738 (513)	total: 25.1s	remaining: 1m 6s
545:	test: 0.8980697	best: 0.8980738 (513)	total: 25.1s	remaining: 1m 6s
546:	test: 0.8980699	best: 0.8980738 (513)	total: 25.2s	remaining: 1m 6s
547:	test: 0.8980688	best: 0.8980738 (513)	total: 25.2s	remaining: 1m 6s
548:	test: 0.8980737	best: 0.8980738 (513)	total: 25.3s	remaining: 1m 6s
549:	test: 0.8980759	best: 0.8980759 (549)	total: 25.3s	remaining: 1m 6s
550:	test: 0.8980825	best: 0.8980825 (550)	total: 25.4s	remaining: 1m 6s
551:	test: 0.8980852	best: 0.8980852 (551)	total: 25.4s	remaining: 1m 6s
552:	test: 0.8980809	best: 0.8980852 (551)	total: 25.4s	remaining: 1m 6s
553:	test: 0.8980802	best: 0.8980852 (551)	total: 25.5s	remaining: 1m 6s
554:	test: 0.8980786	best: 0.8980852 (551)	total: 25.5s	remaining: 1m 6s
555:	test: 0.8980764	best: 0.8980852 (551)	total: 25.6s	remaining: 1m 6s
556:	test: 0.8980785	best: 0.8980852 (551)	total: 2

659:	test: 0.8980915	best: 0.8981166 (579)	total: 30.2s	remaining: 1m 1s
660:	test: 0.8980936	best: 0.8981166 (579)	total: 30.2s	remaining: 1m 1s
661:	test: 0.8980932	best: 0.8981166 (579)	total: 30.3s	remaining: 1m 1s
662:	test: 0.8980939	best: 0.8981166 (579)	total: 30.3s	remaining: 1m 1s
663:	test: 0.8980923	best: 0.8981166 (579)	total: 30.4s	remaining: 1m 1s
664:	test: 0.8980915	best: 0.8981166 (579)	total: 30.4s	remaining: 1m 1s
665:	test: 0.8980917	best: 0.8981166 (579)	total: 30.5s	remaining: 1m 1s
666:	test: 0.8980978	best: 0.8981166 (579)	total: 30.5s	remaining: 1m
667:	test: 0.8980979	best: 0.8981166 (579)	total: 30.6s	remaining: 1m
668:	test: 0.8981036	best: 0.8981166 (579)	total: 30.6s	remaining: 1m
669:	test: 0.8981025	best: 0.8981166 (579)	total: 30.6s	remaining: 1m
670:	test: 0.8981027	best: 0.8981166 (579)	total: 30.7s	remaining: 1m
671:	test: 0.8981063	best: 0.8981166 (579)	total: 30.7s	remaining: 1m
672:	test: 0.8981039	best: 0.8981166 (579)	total: 30.8s	remaining: 1m

777:	test: 0.8981011	best: 0.8981527 (716)	total: 35.3s	remaining: 55.5s
778:	test: 0.8981010	best: 0.8981527 (716)	total: 35.4s	remaining: 55.5s
779:	test: 0.8981007	best: 0.8981527 (716)	total: 35.4s	remaining: 55.4s
780:	test: 0.8981011	best: 0.8981527 (716)	total: 35.5s	remaining: 55.4s
781:	test: 0.8980988	best: 0.8981527 (716)	total: 35.5s	remaining: 55.3s
782:	test: 0.8980995	best: 0.8981527 (716)	total: 35.6s	remaining: 55.3s
783:	test: 0.8980990	best: 0.8981527 (716)	total: 35.6s	remaining: 55.2s
784:	test: 0.8980980	best: 0.8981527 (716)	total: 35.7s	remaining: 55.2s
785:	test: 0.8981007	best: 0.8981527 (716)	total: 35.7s	remaining: 55.1s
786:	test: 0.8980989	best: 0.8981527 (716)	total: 35.7s	remaining: 55.1s
787:	test: 0.8980978	best: 0.8981527 (716)	total: 35.8s	remaining: 55.1s
788:	test: 0.8980904	best: 0.8981527 (716)	total: 35.8s	remaining: 55s
789:	test: 0.8980945	best: 0.8981527 (716)	total: 35.9s	remaining: 55s
790:	test: 0.8980945	best: 0.8981527 (716)	total: 35.9s

894:	test: 0.8980871	best: 0.8981527 (716)	total: 40.5s	remaining: 50s
895:	test: 0.8980882	best: 0.8981527 (716)	total: 40.5s	remaining: 49.9s
896:	test: 0.8980896	best: 0.8981527 (716)	total: 40.6s	remaining: 49.9s
897:	test: 0.8980806	best: 0.8981527 (716)	total: 40.6s	remaining: 49.8s
898:	test: 0.8980824	best: 0.8981527 (716)	total: 40.7s	remaining: 49.8s
899:	test: 0.8980817	best: 0.8981527 (716)	total: 40.7s	remaining: 49.8s
900:	test: 0.8981058	best: 0.8981527 (716)	total: 40.7s	remaining: 49.7s
901:	test: 0.8981075	best: 0.8981527 (716)	total: 40.8s	remaining: 49.7s
902:	test: 0.8981079	best: 0.8981527 (716)	total: 40.8s	remaining: 49.6s
903:	test: 0.8981070	best: 0.8981527 (716)	total: 40.9s	remaining: 49.6s
904:	test: 0.8981075	best: 0.8981527 (716)	total: 40.9s	remaining: 49.5s
905:	test: 0.8981091	best: 0.8981527 (716)	total: 41s	remaining: 49.5s
906:	test: 0.8981058	best: 0.8981527 (716)	total: 41s	remaining: 49.4s
907:	test: 0.8981024	best: 0.8981527 (716)	total: 41.1s	r

0.7936206468568369

In [16]:
from sklearn.metrics import roc_auc_score
param = {                    "objective" : "binary",
                    "metric" : "auc", 
                    "boosting": "gbdt",
                    'n_jobs': 6,
                   'learning_rate': 0.008,
                    'max_cat_to_onehot': 4, 
                     'min_child_samples': int(198.98108530948596),
                    'cat_smooth': 10.0,
                    'n_estimators': int(4530),
                     'max_depth':int(round(2.6)),
                     'num_leaves':int(49),
                     'lambda_l1':max(8.1, 0),
                     'lambda_l2':max(0.8, 0),
                     'min_split_gain':0.0734219488215243,
                     'min_child_weight':41.51557865541733}
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
skf=StratifiedKFold(5,shuffle=True,random_state=315)
val_pr=np.zeros(len(df_train))
feature_importance_df = pd.DataFrame()
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    train_data=lgb.Dataset(df_train[df_train_columns].loc[train_index],label=df_train['outliers'].loc[train_index])
    val_data=lgb.Dataset(df_train[df_train_columns].loc[val_index],label=df_train['outliers'].loc[val_index])
    num_round = 10000
    model=lgb.train(param,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=100,early_stopping_rounds=400)
    fold_importance_df=pd.DataFrame()
    fold_importance_df['feature'] = df_train_columns
    fold_importance_df['importance']=model.feature_importance
    feature_importance_df=pd.concat([feature_importance_df,fold_importance_df],axis=0)
    val_pr[val_index]=model.predict(df_train[df_train_columns].loc[val_index],num_iteration=model.best_iteration)
roc_auc_score(df_train['outliers'],val_pr)

Training until validation scores don't improve for 400 rounds.
[100]	training's auc: 0.903306	valid_1's auc: 0.896589
[200]	training's auc: 0.905026	valid_1's auc: 0.899067
[300]	training's auc: 0.905279	valid_1's auc: 0.899323
[400]	training's auc: 0.905634	valid_1's auc: 0.899068
[500]	training's auc: 0.905873	valid_1's auc: 0.899049
[600]	training's auc: 0.906029	valid_1's auc: 0.899025
Early stopping, best iteration is:
[255]	training's auc: 0.905201	valid_1's auc: 0.899388
Training until validation scores don't improve for 400 rounds.
[100]	training's auc: 0.903217	valid_1's auc: 0.903073
[200]	training's auc: 0.903877	valid_1's auc: 0.904146
[300]	training's auc: 0.903862	valid_1's auc: 0.904079
[400]	training's auc: 0.904182	valid_1's auc: 0.904155
[500]	training's auc: 0.904559	valid_1's auc: 0.904204
[600]	training's auc: 0.9046	valid_1's auc: 0.90428
[700]	training's auc: 0.904747	valid_1's auc: 0.904323
[800]	training's auc: 0.904951	valid_1's auc: 0.904303
[900]	training's 

0.9002243636145087

In [14]:
from sklearn.linear_model import LogisticRegression
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
df_train=df_train.replace([np.inf, -np.inf], np.nan)
df_train=df_train.fillna(0)
skf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
for train_index,val_index in skf.split(df_train,df_train['outliers'].values):
    num_round = 10000
    model = LogisticRegression()
    model.fit(df_train[df_train_columns].loc[train_index],df_train['outliers'].loc[train_index])
    val_pr[val_index]=model.predict_proba(df_train[df_train_columns].loc[val_index])[:,1]
roc_auc_score(df_train['outliers'],val_pr)

0.8964279015628394

In [20]:
df_train

,PROB_CAT_7part_0.89.,PROB_CAT_8part_0.89.,PROB_LGB_6part_0.9.,PROB_LGB_7part_0.9.,PROB_LGB_half_0.9.,PROB_NN_0.8797.,target,outliers
0,0.000544,0.000744,0.000567,0.000509,0.000576,0.000329,-0.82030,0
1,0.011602,0.009733,0.011601,0.014659,0.012816,0.005174,0.39280,0
2,0.003329,0.003291,0.002996,0.003469,0.004034,0.001299,0.68800,0
3,0.000828,0.001130,0.000759,0.001097,0.001193,0.000955,0.14250,0
4,0.000288,0.000244,0.000551,0.000477,0.000570,0.000296,-0.15980,0
5,0.062439,0.037347,0.034799,0.059510,0.038417,0.010584,0.87160,0
6,0.001748,0.001333,0.001652,0.001898,0.001046,0.000497,0.23010,0
7,0.004943,0.004889,0.001284,0.001659,0.001336,0.003624,2.13700,0
8,0.004804,0.004489,0.003382,0.004771,0.006148,0.014688,-0.06540,0
9,0.002011,0.001996,0.001327,0.001210,0.001664,0.000982,0.30000,0


In [12]:
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
df_train_columns=[c for c in df_train.columns if c not in ['target','outliers','card_id']]
def optim_lgb(min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight):
    def run_lgb(train_data,val_data,val_x,min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight):
        num_round = 10000
        params = {
                    "objective" : "binary",
                    "metric" : "binary_logloss", 
                    "boosting": "gbdt",
                    'n_jobs': 6,
                   'learning_rate': 0.008,
                    'max_cat_to_onehot': 4, 
                     'min_child_samples': int(min_child),
                    'cat_smooth': 10.0,
                    'n_estimators': int(n_estimator),
                     'max_depth':int(round(max_depth)),
                     'num_leaves':int(num_leaves),
                     'lambda_l1':max(lambda_l1, 0),
                     'lambda_l2':max(lambda_l2, 0),
                     'min_split_gain':min_split_gain,
                     'min_child_weight':min_child_weight
                }
        model=lgb.train(params,train_data,num_round,valid_sets=[train_data,val_data],verbose_eval=False,early_stopping_rounds=400)

        pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    #    return pred_val
        return  pred_val
    rskf=StratifiedKFold(5,shuffle=True,random_state=315)
    val_pr=np.zeros(len(df_train))
    for train_index,val_index in rskf.split(df_train,df_train['outliers'].values):
        train_data=lgb.Dataset(df_train[df_train_columns].loc[train_index],label=df_train['outliers'].loc[train_index])
        val_data=lgb.Dataset(df_train[df_train_columns].loc[val_index],label=df_train['outliers'].loc[val_index])
        val_pr[val_index]=run_lgb(train_data,val_data,df_train[df_train_columns].loc[val_index],min_child,n_estimator,max_depth,num_leaves,lambda_l1,lambda_l2,min_split_gain,min_child_weight)
    return roc_auc_score(df_train['outliers'],val_pr)
    
from bayes_opt import BayesianOptimization
optimizer = BayesianOptimization(optim_lgb,{'min_child':(5,200),'n_estimator':(50,7000),'max_depth':(-1,20),'num_leaves':(10,50),'lambda_l1':(0,200),'lambda_l2':(0,200),
'min_split_gain':(0.001, 0.1),'min_child_weight':(2,50)}
    
)
optimizer.maximize(init_points=3,
    n_iter=30)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_child | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8984   |  64.23    |  155.2    |  7.35     |  19.38    |  16.71    |  0.06206  |  4.702e+0 |  14.77    |
|  2        |  0.8978   |  82.98    |  156.2    |  5.39     |  133.1    |  30.87    |  0.0315   |  4.309e+0 |  41.87    |
|  3        |  0.8941   |  151.6    |  133.7    |  14.69    |  10.66    |  23.12    |  0.03549  |  695.6    |  27.49    |
|  4        |  0.8929   |  193.2    |  14.05    |  8.928    |  43.64    |  22.08    |  0.04302  |  6.996e+0 |  46.2     |
|  5        |  0.901    |  1.334    |  198.1    |  5.372    |  198.5    |  21.91    |  0.02644  |  6.756e+0 |  25.95    |
|  6        |  0.9007   |  0.2244   |  16.41    |  4.019    |  189.3    |  10.22    |  0.07081  |  3.467e+0 |  19.23    |
|  7        |  0.8908   

In [28]:
path='D:\Python\Elo\Stack'
data=pd.read_csv(os.path.join(path,'prediction_LGB_full_3.64544.csv'))
prediction=pd.DataFrame({'target':df_train['target'],'prob':df_train['PROB_LGB_half_0.9.'],'rmse':data['target']})
prediction

,target,prob,rmse
0,-0.82030,0.000576,-0.309887
1,0.39280,0.012816,-0.631159
2,0.68800,0.004034,0.735281
3,0.14250,0.001193,0.145482
4,-0.15980,0.000570,-0.085556
5,0.87160,0.038417,-1.670417
6,0.23010,0.001046,-0.223420
7,2.13700,0.001336,0.194107
8,-0.06540,0.006148,0.229899
9,0.30000,0.001664,0.143660


In [35]:
rskf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
itog=[]
for train_index,val_index in rskf.split(df_train,df_train['outliers'].values):
    thresholds = []
    for thresh in np.arange(0.01, 1, 0.01):
        thresh = np.round(thresh, 2)
        pr=np.where(prediction['prob'].loc[val_index]>thresh,-33.22,prediction['rmse'].loc[val_index])
        res = np.sqrt(mean_squared_error(pr,prediction['target'].loc[val_index].values))
        thresholds.append([thresh, res])
    thresholds.sort(key=lambda x: x[1], reverse=False)
    best_thresh = thresholds[0][0]
    itog.append(best_thresh)
np.sqrt(mean_squared_error(np.where(prediction['prob']>np.mean(itog),-33.22,prediction['rmse']),prediction['target'].values))

3.645724114953824

In [34]:
np.mean(itog)

0.546